# Seurat object  
## Load required packages


In [ ]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library("scales")
library(harmony)


## Load seurat objects (RAW)



In [ ]:
BT1.diff <- readRDS(file = './results/rds/samples/Filtered_BT1_diff.rds')
BT1.prolif <- readRDS(file = './results/rds/samples/Filtered_BT1_prolif.rds')
BT1.diff@meta.data$line <- "BT1"     
BT1.prolif@meta.data$line <- "BT1"     

BT2.diff <- readRDS(file = './results/rds/samples/Filtered_BT2_diff.rds')
BT2.prolif <- readRDS(file = './results/rds/samples/Filtered_BT2_prolif.rds')
BT2.diff@meta.data$line <- "BT2"     
BT2.prolif@meta.data$line <- "BT2"  

BT54.diff <- readRDS(file = './results/rds/samples/Filtered_BT54_diff.rds')
BT54.prolif <- readRDS(file = './results/rds/samples/Filtered_BT54_prolif.rds')
BT54.diff@meta.data$line <- "BT54"     
BT54.prolif@meta.data$line <- "BT54"  

BT88.diff <- readRDS(file = './results/rds/samples/Filtered_BT88_diff.rds')
BT88.prolif <- readRDS(file = './results/rds/samples/Filtered_BT88_prolif.rds')
BT88.diff@meta.data$line <- "BT88"     
BT88.prolif@meta.data$line <- "BT88"  

LGG85.diff <- readRDS(file = './results/rds/samples/Filtered_LGG85_diff.rds')
LGG85.prolif <- readRDS(file = './results/rds/samples/Filtered_LGG85_prolif.rds')
LGG85.diff@meta.data$line <- "LGG85"     
LGG85.prolif@meta.data$line <- "LGG85"  

LGG275.diff <- readRDS(file = './results/rds/samples/Filtered_LGG275_diff.rds')
LGG275.prolif <- readRDS(file = './results/rds/samples/Filtered_LGG275_prolif.rds')
LGG275.diff@meta.data$line <- "LGG275"     
LGG275.prolif@meta.data$line <- "LGG275" 

LGG336.diff <- readRDS(file = './results/rds/samples/Filtered_LGG336_diff.rds')
LGG336.prolif <- readRDS(file = './results/rds/samples/Filtered_LGG336_prolif.rds')
LGG336.diff@meta.data$line <- "LGG336"     
LGG336.prolif@meta.data$line <- "LGG336" 

LGG349.diff <- readRDS(file = './results/rds/samples/Filtered_LGG349_diff.rds')
LGG349.prolif <- readRDS(file = './results/rds/samples/Filtered_LGG349_prolif.rds')
LGG349.diff@meta.data$line <- "LGG349"     
LGG349.prolif@meta.data$line <- "LGG349" 



## merge raw samples


In [ ]:
seurat_obj <- merge( BT1.prolif, y = c(BT1.diff, BT2.prolif, BT2.diff ,
    BT54.prolif, BT54.diff, BT88.prolif, BT88.diff, LGG85.prolif, 
    LGG85.diff, LGG275.prolif, LGG275.diff, LGG336.prolif, 
    LGG336.diff, LGG349.prolif, LGG349.diff), 
  add.cell.ids = c("BT1.prolif","BT1.diff", "BT2.prolif", "BT2.diff", "BT54.prolif", 
    "BT54.diff", "BT88.prolif", "BT88.diff", "LGG85.prolif", "LGG85.diff", "LGG275.prolif", 
    "LGG275.diff", "LGG336.prolif", "LGG336.diff", "LGG349.prolif", "LGG349.diff"), 
  merge.data = TRUE)

seurat_obj


In [ ]:
rm(BT1.prolif, BT1.diff, BT2.prolif, BT2.diff ,
    BT54.prolif, BT54.diff, BT88.prolif, BT88.diff, LGG85.prolif, 
    LGG85.diff, LGG275.prolif, LGG275.diff, LGG336.prolif, 
    LGG336.diff, LGG349.prolif, LGG349.diff)



##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
pc = 60




# QC metrics (before norm)


In [ ]:
metadata <- seurat_obj@meta.data




## Nb of reads  


In [ ]:
VlnPlot(seurat_obj,features="nCount_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "nCount_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nCount_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()



## Nb of genes  


In [ ]:
VlnPlot(seurat_obj,features="nFeature_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "nFeature_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nFeature_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()
    


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()


## Gene expression  
We can pick the first 100 cells and look at the distributions of their expression values.


In [ ]:
apply(seurat_obj@assays$RNA@data,1,mean) -> gene.expression
sort(gene.expression, decreasing = TRUE) -> gene.expression

as.tibble(
  seurat_obj@assays$RNA@data[,1:100]
) %>%
  pivot_longer(
    cols=everything(),
    names_to="cell",
    values_to="expression"
  ) %>%
  ggplot(aes(x=expression, group=cell)) +
  geom_density() +
  coord_cartesian(ylim=c(0,0.6), xlim=c(0,3))



## MT genes  


In [ ]:
seurat_obj[["percent.mt"]] <- PercentageFeatureSet(seurat_obj, pattern = "^MT-")
VlnPlot(seurat_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()



## Ribosomal genes  


In [ ]:
PercentageFeatureSet(seurat_obj,pattern="^RP[LS]") -> seurat_obj[["percent.ribosomal"]] 
VlnPlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



## Largest gene  


In [ ]:
apply(
  seurat_obj@assays$RNA@counts,
  2,
  max
) -> largest_count

apply(
  seurat_obj@assays$RNA@counts,
  2,
  which.max
) -> largest_index

rownames(seurat_obj)[largest_index] -> seurat_obj$largest_gene

100 * largest_count / seurat_obj$nCount_RNA -> seurat_obj$percent.largest_gene

VlnPlot(seurat_obj,features="percent.largest_gene", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.largest_gene", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## FeatureScatter  


In [ ]:
FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "percent.mt", group.by ="orig.ident")
FeatureScatter(seurat_obj,feature1 = "nCount_RNA", feature2 = "percent.largest_gene",group.by ="orig.ident")

FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by ="orig.ident") + geom_smooth(method = 'lm')



## Cell cycle scoring  


In [ ]:
seurat_obj <- CellCycleScoring(seurat_obj, s.features = cc.genes.updated.2019$s.genes, g2m.features = cc.genes.updated.2019$g2m.genes, set.ident = TRUE)

as_tibble(seurat_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3'))+ geom_bar()


as_tibble(seurat_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))


# Normalization  
## Zero counts   
See features without any counts left after the filtering.


In [ ]:
counts <- GetAssayData(seurat_obj, assay = "RNA")
zero <- which(rowSums(counts) == 0 )
length(zero)

keep.genes <- which(rowSums(counts) != 0 )
seurat_obj <- subset(seurat_obj, features = keep.genes)
seurat_obj


## Normalize
1: Normalizes the feature expression measurements for each cell by the total expression, multiplies this by a scale factor (10,000 by default), and log-transforms the result.  

This simplistic normalisation doesn’t actually normalise the quantitative data very well because it’s so biased by the proportion of zero values in the dataset. This simplistic normalisation therefore doesn’t do a great job in this instance.

2: With a centered log ratio transformation - more similar to the sort of size factor based normalisation which is used for many RNA-Seq experiments.


In [ ]:
seurat_obj <- NormalizeData(seurat_obj, normalization.method = "LogNormalize", scale.factor = 10000)




# QC metrics (after norm) 


In [ ]:
metadata <- seurat_obj@meta.data




## Nb of reads  


In [ ]:
VlnPlot(seurat_obj,features="nCount_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "nCount_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nCount_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()


In [ ]:
nCount_metabo <- as.data.frame(rowSums(seurat_obj[["nCount_RNA"]]))
nCount_metabo$Genes <- "metabo"
colnames(nCount_metabo)[1] <- "nCount"

nCount_all <- as.data.frame(rowSums(seurat_obj[["nCount_RNA"]]))
nCount_all$Genes <- "all"
colnames(nCount_all)[1] <- "nCount"

other <- nCount_all
other$Genes <- "others"
dim(other)
other <- other[!(rownames(other) %in% rownames(seurat_obj$RNA)),]

expr <- rbind(nCount_all, nCount_metabo, other)
head(expr)
ggplot(expr, aes(x=Genes,y=nCount)) + geom_boxplot()



## Nb of genes  


In [ ]:
VlnPlot(seurat_obj,features="nFeature_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "nFeature_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nFeature_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
    scale_x_log10()
    scale_x_log10()
    
  scale_x_log10()
    


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()


## Gene expression  
We can pick the first 100 cells and look at the distributions of their expression values.


In [ ]:
as.tibble(
  seurat_obj@assays$RNA@data[,1:100]
) %>%
  pivot_longer(
    cols=everything(),
    names_to="cell",
    values_to="expression"
  ) %>%
  ggplot(aes(x=expression, group=cell)) +
  geom_density() +
  coord_cartesian(ylim=c(0,0.6), xlim=c(0,3))


In [ ]:
expression_metabo <- as.data.frame(rowSums(seurat_obj@assays$RNA@data))
expression_metabo$Genes <- "metabo"
colnames(expression_metabo)[1] <- "expression"

expression_all <- as.data.frame(rowSums(seurat_obj@assays$RNA@data))
expression_all$Genes <- "all"
colnames(expression_all)[1] <- "expression"

other <- expression_all
other$Genes <- "others"
dim(other)
other <- other[!(rownames(other) %in% rownames(seurat_obj$RNA)),]

expr <- rbind(expression_all, expression_metabo, other)
head(expr)
ggplot(expr, aes(x=Genes,y=log(expression+1))) + geom_boxplot()


## MT genes  
### All genes


In [ ]:
VlnPlot(seurat_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()



### Metabo only


In [ ]:
seurat_obj[["percent.mt"]] <- PercentageFeatureSet(seurat_obj, pattern = "^MT-", assay = 'RNA')
VlnPlot(seurat_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()


## Ribosomal genes  
### All genes  


In [ ]:
VlnPlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



### Metabo only  


In [ ]:
PercentageFeatureSet(seurat_obj,pattern="^RP[LS]", assay = 'RNA') -> seurat_obj[["percent.ribosomal"]] 
VlnPlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



## Largest gene  (All genes)


In [ ]:
VlnPlot(seurat_obj, features = "percent.largest_gene", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.largest_gene", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## FeatureScatter  


In [ ]:
FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "percent.mt", group.by ="orig.ident")

FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by ="orig.ident") + geom_smooth(method = 'lm')



## Cell cycle scoring  


In [ ]:
seurat_obj@meta.data %>%
  group_by(orig.ident,Phase) %>%
  count() %>%
  group_by(orig.ident) %>%
  mutate(percent=100*n/sum(n)) %>%
  ungroup() %>%
  ggplot(aes(x=orig.ident,y=percent, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' )) +
  geom_col() +
  ggtitle("Percentage of cell cycle phases per sample")

as_tibble(seurat_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' ))+ geom_bar()


as_tibble(seurat_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3' )) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))



# Identify high variable features  


In [ ]:
seurat_obj <- FindVariableFeatures(seurat_obj, selection.method = "vst", nfeatures = 3000)



In [ ]:
# identify the 10 most highly variable genes
top10 <- head(VariableFeatures(seurat_obj), 10)

# plot the variable features 
plot <- VariableFeaturePlot(seurat_obj)
LabelPoints(plot = plot, points = head(top10,10), repel = TRUE)


We can plot out a graph of the variance vs mean and highlight the selected genes so we can see whether we think we’re likely to capture what we need.    



In [ ]:
as_tibble(HVFInfo(seurat_obj),rownames = "Gene") -> variance.data

variance.data %>% 
  mutate(hypervariable=Gene %in% VariableFeatures(seurat_obj)
) -> variance.data

variance.data %>% 
  ggplot(aes(log(mean),log(variance),color=hypervariable)) + 
  geom_point() + 
  scale_color_manual(values=c("black","red"))


# Scaling the data  

**ScaleData** :

* Shifts the expression of each gene, so that the mean expression across cells is 0
* Scales the expression of each gene, so that the variance across cells is 1 
    + This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate


Signals separating non-cycling cells and cycling cells will be maintained, but differences in cell cycle phase among proliferating cells (which are often uninteresting), will be regressed out of the data.  


In [ ]:
seurat_obj$CC.Difference <- seurat_obj$S.Score - seurat_obj$G2M.Score

all.genes <- rownames(seurat_obj)
seurat_obj <- ScaleData(seurat_obj, vars.to.regress = "CC.Difference", features = all.genes)


# Linear dimensional reduction  
## PCA  


In [ ]:
seurat_obj <- RunPCA(seurat_obj, verbose = FALSE)
VizDimLoadings(seurat_obj, dims = 1:2, reduction = "pca")


In [ ]:
mat <- Seurat::GetAssayData(seurat_obj, assay = "RNA", slot = "scale.data") ## instead of RNA
pca <- seurat_obj[["pca"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))
eigValues = (pca@stdev)^2  ## EigenValues

varExplained = eigValues / total_variance

pc1 = percent(varExplained[1], accuracy = 0.01)
pc2 = percent(varExplained[2], accuracy = 0.01)


In [ ]:
DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))
DimPlot(seurat_obj, reduction = "pca", group.by = "line") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))
DimPlot(seurat_obj, reduction = "pca", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue',S='tomato3'))

DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident", label.size = 4, label = TRUE)
DimPlot(seurat_obj, reduction = "pca", group.by = "line", label.size = 4, label = TRUE)



### Split by condition


In [ ]:
DimPlot(seurat_obj, reduction = "pca", group.by="orig.ident", split.by = "condition")
DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "pca", group.by="orig.ident", split.by = "condition")+ NoLegend()


In [ ]:
DimPlot(seurat_obj, reduction = "pca", group.by="line", split.by = "condition")
DimPlot(seurat_obj, reduction = "pca", group.by = "line", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "pca", group.by="line", split.by = "condition")+ NoLegend()


# Harmony  
By default, Harmony accepts a normalized gene expression matrix and performs PCA.
*do_pca=FALSE if PCA already done*


In [ ]:
seurat_obj <- RunHarmony(seurat_obj, group.by.vars =c("line", "condition"), reduction = "pca", plot_convergence = TRUE)




## plots  


In [ ]:
mat <- Seurat::GetAssayData(seurat_obj, assay = "RNA", slot = "scale.data")
harmony <- seurat_obj[["harmony"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))
eigValues = (harmony@stdev)^2  ## EigenValues

varExplained = eigValues / total_variance

pc1 = percent(varExplained[1], accuracy = 0.01)
pc2 = percent(varExplained[2], accuracy = 0.01)

DimPlot(seurat_obj, reduction = "harmony", group.by = "orig.ident") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))


In [ ]:
DimPlot(seurat_obj, reduction = "harmony", group.by = "orig.ident")
DimPlot(seurat_obj, reduction = "harmony", group.by = "line")
DimPlot(seurat_obj, reduction = "harmony", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue',S='tomato3'))

DimPlot(seurat_obj, reduction = "harmony", group.by = "orig.ident", label.size = 4, label = TRUE)
DimPlot(seurat_obj, reduction = "harmony", group.by = "line", label.size = 4, label = TRUE)



### Split by condition


In [ ]:
DimPlot(seurat_obj, reduction = "harmony", group.by="orig.ident", split.by = "condition")
DimPlot(seurat_obj, reduction = "harmony", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "harmony", group.by="orig.ident", split.by = "condition")+ NoLegend()


In [ ]:
DimPlot(seurat_obj, reduction = "harmony", group.by="line", split.by = "condition")
DimPlot(seurat_obj, reduction = "harmony", group.by = "line", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "harmony", group.by="line", split.by = "condition")+ NoLegend()



## Save seurat object  


In [ ]:
saveRDS(seurat_obj, file = "./results/rds/Norm_all_samples.rds")



# Determine the dimensionality of the dataset  
how many components should we choose to include?  

**JackStrawPlot**  
*significant* PCs as those who have a strong enrichment of **low p-value**

* dashed line : uniform distribution
* solid curve above dash line : strong enrichments of features with low p-values  

*Cannot run Jackstraw plot on SCT data*  


In [ ]:
seurat_obj <- JackStraw(seurat_obj, num.replicate = 100, dims = pc)
jack.data <- seurat_obj 

seurat_obj <- ScoreJackStraw(seurat_obj, dims = 1:pc)

JackStrawPlot(seurat_obj, dims = 1:pc)


**ElbowPlot**  
ranking of principle components based on the percentage of variance explained by each one 


In [ ]:
DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident")
ElbowPlot(seurat_obj, ndims = pc)
